In [1]:
# !pip3 install spacy --quiet
# !pip3 install tensorflow --quiet
# !pip3 install datasets --quiet
# !pip3 install seqeval --quiet
# !pip3 install wandb --quiet

In [2]:
import numpy as np
import pandas as pd
import spacy
from transformers import AutoTokenizer, AutoModelForTokenClassification
from transformers import pipeline
from transformers import DataCollatorForTokenClassification
from transformers import  TrainingArguments, Trainer
from datasets import load_metric
from datasets import load_dataset
import os
import wandb

/home/jmunse/anaconda3/envs/test/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2022-10-06 16:58:58.585269: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-10-06 16:59:08.993666: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2022-10-06 16:59:09.034679: E tensorflow/stream_executor/cuda/cuda_blas.cc:298

In [4]:

tokenizer = AutoTokenizer.from_pretrained("dslim/bert-base-NER")
model = AutoModelForTokenClassification.from_pretrained("dslim/bert-base-NER")

nlp = pipeline("ner", model=model, tokenizer=tokenizer)
example = "My name is Wolfgang and I live in Berlin"

ner_results = nlp(example)
print(ner_results)

Downloading: 100%|██████████| 59.0/59.0 [00:00<00:00, 24.4kB/s]
Downloading: 100%|██████████| 829/829 [00:00<00:00, 373kB/s]
Downloading: 100%|██████████| 213k/213k [00:00<00:00, 24.3MB/s]
Downloading: 100%|██████████| 2.00/2.00 [00:00<00:00, 1.09kB/s]
Downloading: 100%|██████████| 112/112 [00:00<00:00, 66.2kB/s]
Downloading: 100%|██████████| 433M/433M [00:05<00:00, 80.8MB/s] 


[{'entity': 'B-PER', 'score': 0.9990139, 'index': 4, 'word': 'Wolfgang', 'start': 11, 'end': 19}, {'entity': 'B-LOC', 'score': 0.999645, 'index': 9, 'word': 'Berlin', 'start': 34, 'end': 40}]


In [3]:
dataset = load_dataset("wikiann", "en")
label_names = dataset["train"].features["ner_tags"].feature.names

Dataset wikiann downloaded and prepared to /home/jmunse/.cache/huggingface/datasets/wikiann/en/1.1.0/4bfd4fe4468ab78bb6e096968f61fab7a888f44f9d3371c2f3fea7e74a5a354e. Subsequent calls will reuse this data.


100%|██████████| 3/3 [00:00<00:00, 128.96it/s]


In [6]:
os.environ["WANDB_API_KEY"]="ef0384869fde2f9f620eec955ad6264dec46218c"
os.environ["WANDB_ENTITY"]="Chiemeka"
os.environ["WANDB_PROJECT"]="finetune_bert_ner"

In [5]:
#Get the values for input_ids, token_type_ids, attention_mask
def tokenize_adjust_labels(all_samples_per_split):
  tokenized_samples = tokenizer.batch_encode_plus(all_samples_per_split["tokens"], is_split_into_words=True)
  #tokenized_samples is not a datasets object so this alone won't work with Trainer API, hence map is used 
  #so the new keys [input_ids, labels (after adjustment)]
  #can be added to the datasets dict for each train test validation split
  total_adjusted_labels = []
  print(len(tokenized_samples["input_ids"]))
  for k in range(0, len(tokenized_samples["input_ids"])):
    prev_wid = -1
    word_ids_list = tokenized_samples.word_ids(batch_index=k)
    existing_label_ids = all_samples_per_split["ner_tags"][k]
    i = -1
    adjusted_label_ids = []
   
    for wid in word_ids_list:
      if(wid is None):
        adjusted_label_ids.append(-100)
      elif(wid!=prev_wid):
        i = i + 1
        adjusted_label_ids.append(existing_label_ids[i])
        prev_wid = wid
      else:
        label_name = label_names[existing_label_ids[i]]
        adjusted_label_ids.append(existing_label_ids[i])
        
    total_adjusted_labels.append(adjusted_label_ids)
  tokenized_samples["labels"] = total_adjusted_labels
  return tokenized_samples

tokenized_dataset = dataset.map(tokenize_adjust_labels, batched=True)

  0%|          | 0/10 [00:00<?, ?ba/s]

1000


 30%|███       | 3/10 [00:00<00:00,  7.24ba/s]

1000
1000
1000


 50%|█████     | 5/10 [00:00<00:00,  9.30ba/s]

...

1000


 80%|████████  | 16/20 [00:01<00:00,  7.93ba/s]

1000
1000
1000


100%|██████████| 20/20 [00:02<00:00,  9.68ba/s]

1000
1000
1000


In [6]:
tokenized_dataset

DatasetDict({
    validation: Dataset({
        features: ['tokens', 'ner_tags', 'langs', 'spans', 'input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 10000
    })
    test: Dataset({
        features: ['tokens', 'ner_tags', 'langs', 'spans', 'input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 10000
    })
    train: Dataset({
        features: ['tokens', 'ner_tags', 'langs', 'spans', 'input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 20000
    })
})

In [10]:
tokenized_dataset['train']['ner_tags'][0]

[3, 4, 0, 3, 4, 4, 0, 0, 0, 0, 0]

In [8]:

data_collator = DataCollatorForTokenClassification(tokenizer)

In [9]:
metric = load_metric("seqeval")
def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    # Remove ignored index (special tokens)
    true_predictions = [
        [label_names[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [label_names[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    results = metric.compute(predictions=true_predictions, references=true_labels)
    flattened_results = {
        "overall_precision": results["overall_precision"],
        "overall_recall": results["overall_recall"],
        "overall_f1": results["overall_f1"],
        "overall_accuracy": results["overall_accuracy"],
    }
    for k in results.keys():
      if(k not in flattened_results.keys()):
        flattened_results[k+"_f1"]=results[k]["f1"]

    return flattened_results


/tmp/ipykernel_30109/2854355918.py:1: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric("seqeval")


In [ ]:
#for entity level metrics
flattened_results = {"overall_precision": results["overall_precision"],"overall_recall": results["overall_recall"],"overall_f1": results["overall_f1"],"overall_accuracy": results["overall_accuracy"],}
for k in results.keys():
    if(k not in flattened_results.keys()):
        flattened_results[k+"_f1"]=results[k]["f1"]
      

In [12]:
training_args = TrainingArguments(
    output_dir="./fine_tune_bert_output",
    evaluation_strategy="steps",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=7,
    weight_decay=0.01,
    logging_steps = 1000,
    report_to="wandb",
    run_name = "ep_10_tokenized_11",
    save_strategy='no'
)
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

trainer.train()
wandb.finish()

using `logging_steps` to initialize `eval_steps` to 1000
PyTorch: setting up devices
The following columns in the training set don't have a corresponding argument in `BertForTokenClassification.forward` and have been ignored: spans, tokens, langs, ner_tags. If spans, tokens, langs, ner_tags are not expected by `BertForTokenClassification.forward`,  you can safely ignore this message.
***** Running training *****
  Num examples = 20000
  Num Epochs = 7
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 64
  Gradient Accumulation steps = 1
  Total optimization steps = 2191
Automatic Weights & Biases logging enabled, to disable set os.environ["WANDB_DISABLED"] = "true"


Step,Training Loss,Validation Loss


RuntimeError: Caught RuntimeError in replica 0 on device 0.
Original Traceback (most recent call last):
  File "/home/cokeom/anaconda3/envs/test/lib/python3.9/site-packages/torch/nn/parallel/parallel_apply.py", line 61, in _worker
    output = module(*input, **kwargs)
  File "/home/cokeom/anaconda3/envs/test/lib/python3.9/site-packages/torch/nn/modules/module.py", line 1130, in _call_impl
    return forward_call(*input, **kwargs)
  File "/home/cokeom/anaconda3/envs/test/lib/python3.9/site-packages/transformers/models/bert/modeling_bert.py", line 1759, in forward
    outputs = self.bert(
  File "/home/cokeom/anaconda3/envs/test/lib/python3.9/site-packages/torch/nn/modules/module.py", line 1130, in _call_impl
    return forward_call(*input, **kwargs)
  File "/home/cokeom/anaconda3/envs/test/lib/python3.9/site-packages/transformers/models/bert/modeling_bert.py", line 1015, in forward
    embedding_output = self.embeddings(
  File "/home/cokeom/anaconda3/envs/test/lib/python3.9/site-packag